In [1]:
import numpy as np
from hmmlearn import hmm

In [22]:
priors_     = np.array([0.5,0.5])
transition_ = np.array([[0.95, 0.05],[0.1, 0.9]])
emission_   = np.array([[1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
                       [0.1, 0.1, 0.1, 0.1, 0.1, 0.5]])

In [65]:
class HMM(hmm.MultinomialHMM):
    
    def __init__(self, n_components, priors=None, 
                 transition=None, xemission=None):
        
        super().__init__(n_components)
        
        if ( (priors is not None) & (transition is not None) 
            & (emission is not None) ):
            
            
            self.startprob_ = priors
            self.transmat_ = transition
            self.emissionprob_ = emission
            
            print("Initialized with given A, B and pi")

            
    def smoothing(self, X, t):
        return self.predict_proba(X)[t]    
        
    def score_sequence_prob(self,X):
        return np.exp(self.score(X))

    
    def alpha(self, X):
        pass
    
    def beta(self, X):
        pass
    
    def nu(self, X):
        pass
    


In [66]:
HMM_model = HMM(n_components=2, priors = priors_, transition = transition_, xemission = emission_ )

Initialized with given A, B and pi


In [67]:
 O_seq = np.atleast_2d([5, 0, 5, 2, 5, 5, 5, 5, 1, 5, 5, 4, 5, 2, 0, 3, 1]).T # sequence of observatons

### Problem 1: Find P(O|$\lambda$)

In [71]:
(HMM_model.score(O_seq)) #Log likelihood of sequence of observations given model

-25.284157353270203

In [101]:
HMM_model.score_sequence_prob(O_seq) #likelihood of sequence of observations given model

1.0452736003492322e-11

In [73]:
HMM_model.predict_proba(O_seq) ## State-membership probabilities for each sample in X.

array([[0.08429674, 0.91570326],
       [0.08912551, 0.91087449],
       [0.05818631, 0.94181369],
       [0.05880308, 0.94119692],
       [0.02656823, 0.97343177],
       [0.01688095, 0.98311905],
       [0.01787908, 0.98212092],
       [0.03078485, 0.96921515],
       [0.07140148, 0.92859852],
       [0.07606022, 0.92393978],
       [0.12132934, 0.87867066],
       [0.26264134, 0.73735866],
       [0.3292623 , 0.6707377 ],
       [0.55116884, 0.44883116],
       [0.67063441, 0.32936559],
       [0.72984675, 0.27015325],
       [0.74971595, 0.25028405]])

In [117]:
log_likelihood_O, prob_O = HMM_model.score_samples(O_seq) 
print(log_likelihood_O) #Log likelihood of O.
print(prob_O)  #State-membership probabilities for each sample in X.

-25.284157353270203
[[0.08429674 0.91570326]
 [0.08912551 0.91087449]
 [0.05818631 0.94181369]
 [0.05880308 0.94119692]
 [0.02656823 0.97343177]
 [0.01688095 0.98311905]
 [0.01787908 0.98212092]
 [0.03078485 0.96921515]
 [0.07140148 0.92859852]
 [0.07606022 0.92393978]
 [0.12132934 0.87867066]
 [0.26264134 0.73735866]
 [0.3292623  0.6707377 ]
 [0.55116884 0.44883116]
 [0.67063441 0.32936559]
 [0.72984675 0.27015325]
 [0.74971595 0.25028405]]


In [104]:
HMM_model.smoothing(O_seq,8) # state membership probability of 8th element of the sequence

array([0.07140148, 0.92859852])

### Problem 2: Find optimal state sequence

In [75]:
HMM_model.predict(O_seq) #most probable state sequence for the observations

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [76]:
log_prob, state_seq = HMM_model.decode(O_seq)  
print(log_prob) #Log probability of the produced state sequence
print(state_seq) # most probable state squence for the obsevations

(-27.02964121853847,
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]))

###  Problem 3: Train the model that best fit the data 

In [81]:
X = np.atleast_2d([5, 0, 5, 2, 3, 5, 5, 5, 5, 2, 3, 5, 3, 5, 0, 5, 4, 3, 2, 1, 5, 5, 5, 0, 5, 2, 5, 5, 5, 5, 1, 5, 5, 4, 5, 2, 0, 3, 1, 5, 3, 1, 3, 3, 2, 1, 1, 4, 5, 4, 4, 1, 0, 0, 2, 4, 1, 4, 3, 0, 5, 5, 1, 5, 5, 0, 2, 1, 5, 4, 5, 5, 5, 1, 5, 2, 2, 2, 4, 1, 0, 2, 4, 0, 2, 5, 3, 1, 4, 3, 0, 1, 4, 4, 5, 3, 2, 5, 2, 1, 1, 2, 4, 2, 2, 4, 1, 4, 5, 5, 3, 5, 4, 5, 4, 5, 4, 0, 0, 5, 3, 0, 0, 5, 0, 5, 5, 2, 1, 2, 0, 2, 2, 1, 5, 1, 5, 1, 0, 1, 3, 3, 1, 3, 3, 4, 4, 3, 5, 2, 0, 0, 5, 5, 5, 5, 5, 1, 5, 4, 2, 0, 2, 0, 2, 3, 1, 5, 0, 3, 5, 2, 5, 5, 4, 1, 1, 3, 3, 5, 0, 1, 2, 3, 3, 4, 3, 3, 1, 3, 5, 5, 4, 0, 3, 5, 5, 5, 1, 0, 4, 2, 1, 3, 0, 4, 0, 2, 1, 1, 2, 5, 2, 2, 2, 1, 1, 3, 3, 1, 1, 3, 0, 0, 4, 5, 4, 4, 5, 2, 4, 5, 2, 3, 1, 4, 2, 2, 0, 3, 5, 3, 5, 3, 1, 0, 4, 1, 1, 5, 1, 3, 2, 0, 3, 0, 0, 0, 3, 1, 3, 0, 4, 3, 4, 0, 2, 1, 0, 3, 2, 5, 5, 5, 1, 5, 1, 5, 1, 4, 4, 2, 1, 3, 3, 2, 2, 0, 5, 5, 5, 3, 5, 3, 5, 5, 5, 0, 1, 5]).T

In [106]:
HMM_f_model =  HMM(n_components=3) 
HMM_f_model.fit(X) ## fit a 3-state HMM on data X

C:\Users\Chema Camacho\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


HMM(n_components=3)

In [109]:
HMM_f_model.transmat_ #transition matrix of the fitted model

array([[0.31865605, 0.30286052, 0.37848343],
       [0.24866937, 0.39792277, 0.35340786],
       [0.26944913, 0.41868907, 0.31186181]])

In [110]:
HMM_f_model.emissionprob_ #emission probabilities of the fitted model 

array([[0.11926812, 0.13967781, 0.30872685, 0.22861875, 0.08215298,
        0.12155549],
       [0.11304132, 0.10564188, 0.04748165, 0.19635751, 0.21197618,
        0.32550145],
       [0.17625603, 0.24511739, 0.12696216, 0.056373  , 0.05897747,
        0.33631395]])

In [114]:
HMM_f_model.startprob_ #initial probabilities of the fitted model

array([5.66898483e-05, 4.06691389e-01, 5.93251921e-01])

In [118]:
HMM_f_model.get_stationary_distribution() #stationary distribution of states

array([0.27511197, 0.37895378, 0.34593425])

### Sample from the model

In [115]:
obs_sequence, state_sequence = (HMM_f_model.sample(n_samples=12, random_state=20)) ### observations ### state sequence

In [116]:
print(obs_sequence); print(state_sequence)

[[5]
 [5]
 [3]
 [4]
 [1]
 [5]
 [5]
 [0]
 [3]
 [1]
 [5]
 [1]]
[2 2 0 1 2 1 2 2 0 0 2 1]
